# 1. Importing Packages

In [ ]:
import os
import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping, TensorBoard
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import Recall, Precision

In [ ]:

#Upload The Following Files
from unet import build_unet
from customMetrics import dice_loss, dice_coef, iou

# 2. Setting The Environment

In [ ]:
H = 128
W = 128

In [ ]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

""" Hyperparameters """
batch_size = 8

lr = 1e-4
num_epochs = 100
model_path = os.path.join("/content/files128", "model128.h5")
csv_path = os.path.join("/content/files128", "data.csv")

In [ ]:
""" Dataset """
# Make The Folder and upload the data
dataset_path = "/content/gamma_data_128"
train_path = os.path.join(dataset_path, "train")
valid_path = os.path.join(dataset_path, "valid")

In [ ]:
pwd

'/content'

# 3. Utils Function

In [ ]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [ ]:
def load_data(path):
    print(path)
    x = sorted(glob(os.path.join(path, "image", "*.jpg")))
    y = sorted(glob(os.path.join(path, "gt", "*.jpg")))
    return x, y

In [ ]:
def read_image(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_COLOR)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    return x

In [ ]:
def read_mask(path):
    path = path.decode()
    x = cv2.imread(path, cv2.IMREAD_GRAYSCALE)  ## (512, 512)
    # x = cv2.resize(x, (W, H))
    x = x/255.0
    x = x.astype(np.float32)
    x = np.expand_dims(x, axis=-1)              ## (512, 512, 1)
    return x

In [ ]:
def tf_parse(x, y):
    def _parse(x, y):
        x = read_image(x)
        y = read_mask(y)
        return x, y

    x, y = tf.numpy_function(_parse, [x, y], [tf.float32, tf.float32])
    x.set_shape([H, W, 3])
    y.set_shape([H, W, 1])
    return x, y

In [ ]:
def tf_dataset(X, Y, batch_size=2):
    dataset = tf.data.Dataset.from_tensor_slices((X, Y))
    dataset = dataset.map(tf_parse)
    dataset = dataset.batch(batch_size)
    dataset = dataset.prefetch(4)
    return dataset

In [ ]:
train_x, train_y = load_data(train_path)
valid_x, valid_y = load_data(valid_path)

print(f"Train: {len(train_x)} - {len(train_y)}")
print(f"Valid: {len(valid_x)} - {len(valid_y)}")

/content/gamma_data_128/train
/content/gamma_data_128/valid
Train: 1890 - 1890
Valid: 630 - 630


In [ ]:
pwd

'/content'

In [ ]:
create_dir("/content/files128")

In [ ]:
train_dataset = tf_dataset(train_x, train_y, batch_size=batch_size)
valid_dataset = tf_dataset(valid_x, valid_y, batch_size=batch_size)

train_steps = len(train_x)//batch_size
valid_steps = len(valid_x)//batch_size

if len(train_x) % batch_size != 0:
    train_steps += 1
if len(valid_x) % batch_size != 0:
    valid_steps += 1

In [ ]:
train_steps, valid_steps

(237, 79)

# 4. The Model

In [ ]:
""" Model """
model = build_unet((H, W, 3))
model.compile(loss=dice_loss, optimizer=Adam(lr), metrics=[dice_coef, iou, Recall(), Precision()])
model.summary()

Model: "UNET"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
conv2d_19 (Conv2D)              (None, 128, 128, 64) 1792        input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_18 (BatchNo (None, 128, 128, 64) 256         conv2d_19[0][0]                  
__________________________________________________________________________________________________
activation_18 (Activation)      (None, 128, 128, 64) 0           batch_normalization_18[0][0]     
_______________________________________________________________________________________________

In [ ]:
callbacks = [
    ModelCheckpoint(model_path, verbose=1, save_best_only=True),
    ReduceLROnPlateau(monitor="val_loss", factor=0.1, patience=5, min_lr=1e-6, verbose=1),
    CSVLogger(csv_path),
    TensorBoard(),
    EarlyStopping(monitor="val_loss", patience=10, restore_best_weights=False)
]

In [ ]:
model.fit(
    train_dataset,
    epochs=num_epochs,
    validation_data=valid_dataset,
    steps_per_epoch=train_steps,
    validation_steps=valid_steps,
    callbacks=callbacks
)

Epoch 1/100
237/237 [==============================] - 20s 86ms/step - loss: 0.0628 - dice_coef: 0.9374 - iou: 0.8832 - recall_1: 0.4869 - precision_1: 0.9834 - val_loss: 0.2837 - val_dice_coef: 0.7166 - val_iou: 0.5627 - val_recall_1: 0.3459 - val_precision_1: 0.8947

Epoch 00001: val_loss did not improve from 0.27606
Epoch 2/100
237/237 [==============================] - 20s 84ms/step - loss: 0.0621 - dice_coef: 0.9380 - iou: 0.8844 - recall_1: 0.4871 - precision_1: 0.9837 - val_loss: 0.2837 - val_dice_coef: 0.7165 - val_iou: 0.5626 - val_recall_1: 0.3458 - val_precision_1: 0.8949

Epoch 00002: val_loss did not improve from 0.27606
Epoch 3/100
237/237 [==============================] - 20s 84ms/step - loss: 0.0614 - dice_coef: 0.9387 - iou: 0.8855 - recall_1: 0.4873 - precision_1: 0.9841 - val_loss: 0.2838 - val_dice_coef: 0.7164 - val_iou: 0.5625 - val_recall_1: 0.3456 - val_precision_1: 0.8949

Epoch 00003: val_loss did not improve from 0.27606
Epoch 4/100
237/237 [================